Практическое задание 1: Линейная регрессия, метод градиентного спуска

1) Найти данные, на которых интересно будет решать задачу регрессии. Зависимость целового признака от нецелевых должна быть не слишком сложной, чтобы обученная линейная модель смогла показать приемлимый результат.
 - в данном случае взята база данных прочности бетона в зависимости от концентрации входяших в него компонент.
 Источник - https://archive.ics.uci.edu/ml/datasets/concrete+compressive+strength

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

raw_data = pd.read_excel("data/Concrete_Data.xls")
raw_data.head()

Нецелевые параметры (все количественные)
Cement (component 1) -- kg in a m3 mixture 
Blast Furnace Slag (component 2) -- kg in a m3 mixture
Fly Ash (component 3) -- kg in a m3 mixture
Water (component 4) -- kg in a m3 mixture
Superplasticizer (component 5) -- kg in a m3 mixture
Coarse Aggregate (component 6) -- kg in a m3 mixture
Fine Aggregate (component 7) -- kg in a m3 mixture
Age -- Day (1~365)  

Целевой параметр
Concrete compressive strength -- MPa 

2) Считать данные, выполнить первичный анализ данных, при необходимости произвести чистку данных
- проверим данные на наличие пропусков
- переименуем заголовки столбцов для удобства

In [ ]:
raw_data.isnull().sum()

In [ ]:
data = raw_data.set_axis(['Cement', 'Slag', 'Ash', 'Water', 'Superplasticizer','Coarse', 'Fine', 'Age', 'Strength'], axis=1, copy=True)
data.describe()

Из вывода видно, что в данных нет пропусков. Также можно отметить следующие особенности:
* более четверти образцов вообще не имеют шлака или суперпластификатора, а более половины - золы.
* более 75% образцов не старше 2 месяцев.
Это может негативно сказаться на обучении модели, особенно в случае кросс-валидации. Так как в выборки могут попасть данные только с тривиальными значениями в конкретной колонке. Стоит учитывать этот момент при разделени на тестовую и обучающую выборки.

3) Выполнить разведочный анализ (EDA), использовать визуализацию, сделать выводы, которые могут быть полезны при дальнейшем решении задачи регрессии.

In [ ]:
plt.scatter(x=data['Cement'] , y=data['Strength'])

Из графика выше видно, что количесвто цемента (ожидаемо) является показательной характеристикой прочности бетона. При массе меньше 300, невозможно достичь наивысшего качества, подбором других параметров.

In [ ]:
plt.scatter(x=data['Coarse'] , y=data['Fine'], c=data['Strength'])

Из данного графика видно отсутсвие прямой зависимости между наполнителями.

4) При необходимости выполнить полезные преобразования данных (например, трансформировать категариальные признаки в количественные), убрать ненужные признаки, создать новые (Feature Engineering).
- при попытках составить новые признаки, видимые линейные зависимости не наблюдались.

Рассмотрим матрицу попарной корреляции для признаков

In [ ]:
import seaborn
seaborn.heatmap(data.corr(), cbar = True , annot=True)

5) Перед обучением моделей подобрать наилучшее количество (и само подмножество) признаков, например используя Recursive Feature Elimination (RFE).
* зададимся целью убрать самый незначимый параметр. Сделаем это с помощью RFE. Укажем целевое количество параметров на 1 меньше текущего.
Как видим Крупный Наполнитель (Coarse) оказался наименее полезным параметром для линейной регрессии с точки зрения этого алгоритма.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE

x = data.drop(['Strength'], axis=1)
y = data['Strength']

rfe = RFE(estimator=LinearRegression(), n_features_to_select=7)
rfe.fit(x, y)
x = rfe.transform(x)
rfe.get_feature_names_out()

6) Случайным образом разбить данные на обучающую и тестовую выборки, используя методы существующих библиотек.
- тестовая выборка 25%

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

7) При обучении моделей использовать масштабирование данных.
- в случае линейной регрессии очень важный пункт, так как параметры входят как слагаемые в одно выражение.
- воспользуемся стандартными средствами

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train) # запоминаем и примееняем масштаб на тренировочных данных
x_test = scaler.transform(x_test) # применяем к тестовым. То есть они могут выходить за пределы

8) Обучить модель на обучающей выборке, используя функцию gradient_descent(X, y). Оценить качество модели на обучающей и тестовой выборках, используя MSE, RMSE и R^2.

In [ ]:
from functions import gradient_descent
w = gradient_descent(x_train, y_train, alpha=0.05, max_iteration_num=1000)
y_ans_my_grad = x_test.dot(w[1:])+ w[0]

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
from prettytable import PrettyTable

results = PrettyTable(["Exp", "MSE", "RMSE", "R^2"])
results.add_row(["My gradient",  mean_squared_error(y_test, y_ans_my_grad), np.sqrt(mean_squared_error(y_test, y_ans_my_grad)), r2_score(y_test, y_ans_my_grad)])

In [ ]:
from sklearn.linear_model import Ridge
model = Ridge()
model.fit(x_train, y_train)
y_ans_Ridge = model.predict(x_test)
results.add_row(["Ridge",  mean_squared_error(y_test, y_ans_Ridge), np.sqrt(mean_squared_error(y_test, y_ans_Ridge)), r2_score(y_test, y_ans_Ridge)])

9) Повторить тоже самое, но используя кросс-валидацию.
- так как тестовая выборка составляет 25% от общего числа данных, а их колличество не так велико, разобьем тренировочную выборку на 4
- в качестве модели используем LinearRegression(), а для 

In [29]:
from sklearn.model_selection import cross_val_score
cross_scores = cross_val_score(LinearRegression(), x_train, y_train, cv=4, scoring='neg_mean_squared_error')
cross_scores_r2 = cross_val_score(LinearRegression(), x_train, y_train, cv=4, scoring='r2')
results.add_row(["Cross validation", abs(cross_scores.mean()), np.sqrt(abs(cross_scores.mean())), cross_scores_r2.mean()])

In [30]:
print(results)

+------------------+---------------------+--------------------+---------------------+
|       Exp        |         MSE         |        RMSE        |         R^2         |
+------------------+---------------------+--------------------+---------------------+
|   My gradient    |  112.82255504713623 | 10.621796225080589 |  0.5634143251620491 |
|      Ridge       |  112.78934280258935 | 10.620232709436708 |  0.5635428454760265 |
| Cross validation |  0.6090410428395059 | 0.7804108167109846 |          -          |
| Cross validation | -109.38656138359136 |        nan         |          -          |
| Cross validation |  109.38656138359136 | 10.458803056927277 |          -          |
| Cross validation |  109.38656138359136 | 10.458803056927277 |  0.6090410428395059 |
| Cross validation |  109.38656138359136 | 10.458803056927277 |  0.6090410428395059 |
| Cross validation |  138.30276967853203 | 11.760219797203284 | 0.39652842891836737 |
| Cross validation |  109.38656138359136 | 10.45880305